### PROJEKT KOŃCZĄCY BOOTCAMP DATA SCIENCE 2020
### Temat 3: Klasyfikacja wydźwięku twittów
### Autor: Kinga Jamróz

Cel projektu: zastosowanie modeli klasyfikacji do rozpoznawania wydźwięku (pozytywny lub negatywny) twittów.

### 0. Zaimportowanie wszystkich wykorzystanych bibliotek i opcji

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk
import re

from nltk.corpus import stopwords
#nltk.download('words')
from nltk.corpus import words
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

from nltk import FreqDist
from num2words import num2words

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
#from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
#from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC

from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import xgboost as xgb

from sklearn.decomposition import TruncatedSVD

#opcja żeby lepiej widzieć zawartość kolumny z tekstem
pd.set_option('display.max_colwidth', -1)

C:\Users\FINA\anaconda3\lib\site-packages\ipykernel_launcher.py:39: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


### 1. Wczytanie danych

Dane zawierają zdania pochodzące z recenzji produktów, filmów oraz restauracji. Oznaczone są jako pozytywne (wartość 1) lub negatywne (wartość 0).

Dane na których opiera się ponizsza analiza pochodzą z trzech różnych źródeł (trzy osobne pliki). Po ich załadowaniu zostaną połączone i traktwane jako jeden zbiór (zgodnie z poleceniem zadania).

In [2]:
data_amazon=pd.read_csv("data/sentiment labelled sentences/amazon_cells_labelled.txt",delimiter="\t", header=None,
                   names=["text", "label"],encoding='utf-8')
print(data_amazon.shape)
data_amazon.head(3)

(1000, 2)


,text,label
0,So there is no way for me to plug it in here in the US unless I go by a converter.,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1


In [3]:
data_yelp=pd.read_csv("data/sentiment labelled sentences/yelp_labelled.txt",delimiter="\t", header=None,
                   names=["text", "label"],encoding='utf-8')
print(data_yelp.shape)
data_yelp.head(3)

(1000, 2)


,text,label
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0


Ze względu na problemy z plikiem imdb_labelled.txt został on pobrany w inny sposób niż poprzednie. 
Zgodnie z opisem danych wszystkie pliki powinny zawierać 1000 obserwacji (co zgadza się dla pliku yelp_labelled.txt oraz amazon_cells_labelled.txt).
Plik pochodzący z portalu imdb.com nie zaczytuje się w całości (co prezentuję poniżej).
Przyczyną tego jest cudzysłów, który w niektórych wierszach nie jest dokmnięty.
Dlatego też postanowiłam usunąć go i zapisać plik jako nowy aby móc powierać go w sposób jak wyżej.

In [4]:
data_imdb=pd.read_csv("data/sentiment labelled sentences/imdb_labelled.txt",delimiter="\t", header=None,
                   names=["text", "label"],encoding='utf-8')
print(data_imdb.shape)
data_imdb.head(3)

(748, 2)


,text,label
0,"A very, very, very slow-moving, aimless movie about a distressed, drifting young man.",0
1,"Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.",0
2,"Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.",0


###### Zamiana znaków: cudzysłów =  spacja oraz utworzenie nowego pliku imdb_labelled_new.txt. 

Zostało to wykonane przy użyciu poniższego kodu. 
Jest on zakomendowany, w celu uniknięcia nadpisania/zdublowania, gdyż poprawiony plik znajduje się już w lokalizacji 'data\\sentiment labelled sentences' skąd jest pobierany w kolejnym kroku.

In [5]:
# doc = []
# x = open("data\\sentiment labelled sentences\\imdb_labelled_new.txt", "a")
# with open ('data\\sentiment labelled sentences\\imdb_labelled.txt', "r") as f:
#     lines = f.readlines()
#     for line in lines:
#         line_new = line.replace('"', '')
#         doc.append(line_new)
#         x.write(line_new)
# x.close()

In [6]:
#zaczytanie tego naprawionego pliku 
data_imdb_new=pd.read_csv("data\\sentiment labelled sentences\\imdb_labelled_new.txt", delimiter="\t", 
                           header=None, names=["text", "label"],encoding='utf-8')
print(data_imdb_new.shape)
data_imdb_new.head(3)

(1000, 2)


,text,label
0,"A very, very, very slow-moving, aimless movie about a distressed, drifting young man.",0
1,"Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.",0
2,"Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.",0


Połączenie wszystkich wczytanych zbiorów w jeden.

In [7]:
data_all=pd.concat([data_amazon, data_yelp, data_imdb_new], ignore_index=True)
print(data_all.shape)
data_all.head(3)

(3000, 2)


,text,label
0,So there is no way for me to plug it in here in the US unless I go by a converter.,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1


### 2. Przygotowanie danych


#### 2.1 Czyszczenie + normalizacja (stemming)

In [8]:
df=data_all.copy()

Wstępne przygotowanie danych przebiegło w następujących krokach:
* Usunięcie znaków dziwnie zakodowanych (żadne kodowanie podczas importu nie pomogło)
* Zamiana wszystkich liter na małe
* Zamiana symboli emotikon na słowa np.: :) - smile, :( - sadness
* Zamiana wszystkich znaków interpunkcyjnych oraz znaków specjalnych na spacje
* Usunięcie napisów składajacych się z liter i liczb, czyli np. modeli telefonów
* Zakodowanie liczb na słowa (liczby dość często występują w tych tekstach opisując np. przydzielone punkty np. 10/10). (Uwaga: Po takim zakodowaniu słow konieczne było ponowne usunięcie z tekstu myślników)
* Poprawa wyrazów w których znajdują się zduplikowane litery. Taki zapis zapewne może wyrażać pewne emocje, jednak są one trudne do zdefiniowania. Wyrazy poprawiono w taki sposób aby kwalifikowały się do słów występujących w słowniku angielskich zwrotów (np. "WAAAAAAyyyyyyyyyy" na "way")
* Usunięcie słów nie wnoszących nic do analizy (tzw. stopwords)
* Usunięcie pojedynczych liter występujących w tekście
* Zastosowanie stemming'u, czyli usunięcie końcówek występujących w angielskich słowach takich jak "ing", "ly" itp.
    
Uwagi:

* W tekście nie znaleziono adresów email, adresów stron internetowych, czy odnośników to użytkowników (np. @username) dlatego też nie ma kodu usuwajacego takie ciągi znaków (jedynie znaki charakterystyczne: #, @)
* 'Usunięcie' jakichś symboli w większości przypadków oznaczało zamianę ich na spację. To mogło stworzyć w tekście nadmiarowe spacje, jednak w późniejszych przekształceniach nie będą one przeszkadzały ze względu na późniejsze tokenizowanie. 

Poniżej są definicje funkcji pomocniczych do przygotowywania danych. Są one później używane w funkcji preparing_data.

In [9]:
#Usuwanie błędnie zwielokrotnionych liter w wyrazach
def remove_duplicate_letters(input_text): 
    
    """
    Usuwanie błędnie zwielokrotnionych liter w wyrazach
Funkcja, przechodząc wyraz po wyrazie, wyszukuje czy w danym wyrazie wystepuja powtarzajace się litery
Jeżeli ich nie ma,to wyraz się nie zmienia
Jeżeli są to:
    ze wzgledu na fakt, ze w języku angielskim istnieją poprawne wyrazy, które mają dwie takie same litery
    obok siebie (np. seem)
    sprawdza czy wyraz, gdzie będą dwie takie litery należy do słownika angielskich słów
    jesli tak, to takie słowo zostaje zapisane
    jeśli nie, to usuwane są wszytskie zwielokrotnione litery (oprócz oryginalnej)
    
Uwaga: w przypadku, gdy podwojona litery wynika ze zmiany gramatycznej wyrazu (np. dropped) takie słowo nie znajduje się w słowniku words.words()
zatem funkcja zwróci droped. Ze względu na fakt, że w późniejszych etapach będzie używana lematyzacja nie ma to znaczenia.

    """
    duplic_free_text=" "
    wordss = input_text.split()
    pattern = r'(.)\1{1,}'
    for word in wordss:
        duplic=re.search(pattern, word)
        if duplic!=None:
            word_two_char=re.sub(pattern, r'\1\1', word)
            if word_two_char in words.words():
                duplic_free_text = duplic_free_text+" " + word_two_char
            else:
                word_one_char=re.sub(pattern, r'\1', word)
                duplic_free_text = duplic_free_text +" " + word_one_char
        else:
            duplic_free_text = duplic_free_text + " " + word
    return duplic_free_text

In [10]:
#Usuwanie napisów alfanumerycznych, czyli np. modeli telefonów

def remove_alphanumeric(input_text): 
    
    """
Usuwanie napisów alfanumerycznych, czyli np. modeli telefonów, różnego rodzaju kodów zawierających zarówno z liter jak i liczb.
Czyli eliminuje napisy, które nie są wyłącznie literami (isalphanumeric()=False) oraz nie są wyłacznie liczbami (isnumeric()=False)

    """
    alphanumeric_free_text=""
    wordss = input_text.split()
    for word in wordss:
        if word.isnumeric()==True or word.isalpha()==True:
            alphanumeric_free_text = alphanumeric_free_text+" " + word

    return alphanumeric_free_text

In [11]:
#Zamiana liczb na ich odpowiednik słowny

def _conv_num(match):
    return num2words(match.group())

def numbers_to_words(text):
    return re.sub(r'\d+', _conv_num, text)

In [12]:
#Usuwanie szumu, czyli słów, które nic nie wnoszą do analizy (tzw. stopwords)
#lista takich słów, której będe używać, jest zaimportowana na początku skryptu

stopwords_list = stopwords.words('english')

def remove_stopwords(input_text, stopwords_list):
    words = input_text.split() 
    noise_free_words = [word for word in words if word not in stopwords_list] 
    noise_free_text = " ".join(noise_free_words) 
    return noise_free_text

In [13]:
#Stemming: usuwanie końcówek (“ing”, “ly”, “es”, “s” etc) oparte na regułach.

#CONNECTIONS------> CONNECT
#CONNECTED------> CONNECT
#CONNECTING------> CONNECT
#CONNECTION------> CONNECT
stemmer = PorterStemmer()
def stemming(doc):
    stemmed_doc = []
    for line in doc:
        stemmed_doc.append(" ".join([stemmer.stem(x) for x in line.split(" ")]))
    return stemmed_doc

In [14]:
def preparing_data(data_input):
    

    #Znaki, których nie udało się odkodować zamieniam na spację 
    data_input = [re.sub(r'\x97|\x96', " ", x) for x in data_input]
    
    #zamiana na małe litery
    data_input = [str(x).lower() for x in data_input]
    
    #emotikony sadness + smile - zamiana znaków na text
    data_input = [re.sub(r':\) |:-\)|;\)|;-\)', " smile ", x) for x in data_input]
    data_input = [re.sub(r':\(|:-\(', " sadness ", x) for x in data_input]
    
    #znaki interpunkcyjne i różne znaki specjalne - zamiana na spację
    data_input = [re.sub(r'[~\'`!@#$%^&*()_\-+={[|\:;"\<,>.?/\]}]', " ", x) for x in data_input]
    
    #Usuwanie napisów alfanumerycznych, czyli np. modeli telefonów
    data_input = [remove_alphanumeric(row) for row in data_input]

    #Usuwanie błędnie zwielokrotnionych liter w wyrazach
    data_input = [remove_duplicate_letters(row) for row in data_input]
    
    #Zamiana liczb na słowa
    data_input = [numbers_to_words(x) for x in data_input]
    #Ponownie trzeba usunąc myślniki bo liczby pojawiaja sie z myslnikami
    data_input = [re.sub(r'[-]', " ", x) for x in data_input]
    
    #Usuwanie stopwordsów
    data_input = [remove_stopwords(row, stopwords_list) for row in data_input]
    
    #Usunięcie pojedynczych liter
    data_input = [re.sub(r'\s+[a-zA-Z]\s+', "", x) for x in data_input]
    
    data_input = stemming(data_input)
    
    return data_input

In [15]:
df["text"] = preparing_data(df["text"])

In [16]:
df.to_csv('df_clear.csv') 

In [17]:
df_clear=df.copy()

Poniżej kilka przykładów jak zmienił się tekst po wstępnym przygotowaniu danych.

In [18]:
df["text"][[345, 97, 476]]

345    drope phone time say even concret phone still great knock wood
97     found product way big                                         
476    uncomfort ear use lg env                                      
Name: text, dtype: object

In [19]:
data_all["text"][[345, 97, 476]]

345    I've dropped my phone more times than I can say, even on concrete and my phone is still great (knock on wood!).
97     I found this product to be waaay too big.                                                                      
476    Uncomfortable In the Ear, Don't use with LG VX9900 (EnV).                                                      
Name: text, dtype: object

#### 2.2 Reprezentacja tekstu + redukcja wymiaru

In [21]:
# X = features, y=labels
X=df.drop("label", axis=1)
y=df["label"]

In [22]:
#Podział na train i test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Poniżej przekształcenie analizowanych danych do postaci macierzy TF–IDF.

In [24]:
cv = CountVectorizer()
X_cv = cv.fit_transform(X_train["text"])
transformer = TfidfTransformer()
tfidf_corpus = transformer.fit_transform(X_cv)

corpus_array = tfidf_corpus.toarray()
#corpus_array
X_train_tfidf = pd.DataFrame(corpus_array, columns=cv.get_feature_names())
print(X_train_tfidf.shape)
X_train_tfidf.head(3)

(2400, 3552)


,abandon,abhor,abil,abl,abound,abroad,absolut,absolutley,abysm,ac,...,young,youth,yucki,yukon,yummi,yun,zero,zilion,zombi,zombiez
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.422856,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
Xtest_cv = cv.transform(X_test["text"])
tfidf_corpus = transformer.transform(Xtest_cv)
corpus_array = tfidf_corpus.toarray()
#corpus_array
X_test_tfidf = pd.DataFrame(corpus_array, columns=cv.get_feature_names())
print(X_test_tfidf.shape)
X_test_tfidf.head(3)

(600, 3552)


,abandon,abhor,abil,abl,abound,abroad,absolut,absolutley,abysm,ac,...,young,youth,yucki,yukon,yummi,yun,zero,zilion,zombi,zombiez
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
liczba_kolumn = len(X_train_tfidf.columns)
liczba_kolumn

3552

Przed przystąpieniem do modelowania zostanie zmniejszony wymiar analizowanych danych przy użyciu metody SVD.

In [28]:
# obliczenie svd testowo dla 2000 komponentów
svd = TruncatedSVD(n_components=2000)
svd.fit(X_train_tfidf)
#print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum())

0.999502734037476


Macierz TF-IDF ma 3552 kolumn. Podając SVD 2000 komponentów możemy wyjaśnić 99.95% wariancji.

In [29]:
var_explained = svd.explained_variance_ratio_
print(var_explained[var_explained>0.0001].sum())
print(len(var_explained[var_explained>0.0001]))

0.9761373827305844
1555


Natomiast zmniejszenie komponentów o prawie połowę wyjaśnia 97.61%. Dlatego zdecydowałam się użyć SVD o 1555 komponentach.

In [30]:
# obliczenie svd
svd = TruncatedSVD(n_components=1555)
svd.fit(X_train_tfidf)
#print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum())

0.9756548916345562


In [31]:
X_train_svd = pd.DataFrame(svd.transform(X_train_tfidf))
X_test_svd = pd.DataFrame(svd.transform(X_test_tfidf))

In [32]:
print(X_train_svd.shape)
print(X_test_svd.shape)

(2400, 1555)
(600, 1555)


#### 3. Modelowanie

In [33]:
models = [
    LogisticRegression(),
#    DecisionTreeClassifier(),
#    GaussianNB(),
    SVC(),
#    LinearSVC(),
    RandomForestClassifier()
#    xgb.XGBRegressor()
]

In [34]:
params = [
# dla regresji logistycznej rozważamy różne wartości parametru C\lambda (siła regularyzacji) oraz różne warianty regularyzacji 
    {"C":[0.01, 0.1, 1, 10, 100],
     "penalty":["l2", "l1"]},
    
# dla drzewa decyzyjnego sprawdze kilka wariantów parametrów określających kryterium podzialu,
# minimalna wielkosc liscia, liczbe zmiennych bioracych udzial w podziale oraz maksymalna glebokosc drzewa
#     {"criterion":['gini', 'entropy'],
#      "min_samples_leaf":[ 10, 15, 25, 50],
#      "max_features":['auto', 'sqrt', 'log2'],
#      "max_depth":[None, 5, 10, 20]},
    
#Gaussian Naive Bayes - bez optymalizacji parametrow    
#    {},
    
#Support Vector Classification  - różne typy jądra wraz ze współczynnikami oraz rózne wartości parametru regularyzacji,      
    [{"kernel":["rbf"], 
      "gamma":[0.1,1,10], #[0.1,1,10,'scale', 'auto' ]
      "C":[ 0.1, 1, 10]}, #[0.01, 0.1, 1, 10, 100]
     {"kernel":["poly"], 
      "degree":[2,3],
      "C":[ 0.1, 1, 10]}],
    
#LinearSVC - typ oraz siła regularyzacji   
#     {"C":[0.01, 0.1, 1, 10, 100]},
    
# dla lasu losowego sprawdze kilka wariantów liczby drzew, parametrów określających kryterium podzialu,
# minimalna wielkosc liscia, liczbe zmiennych bioracych udzial w podziale oraz maksymalna glebokosc drzewa
    {"n_estimators":[50, 100, 200],
     "criterion":['gini', 'entropy'],
     "min_samples_leaf":[ 10, 25, 50],
     "max_features":['auto', 'sqrt', 'log2'],
     "max_depth":[None, 5, 10, 20]}  
    
#    {"n_estimators":[10, 50, 100],
#     "learning_rate":[0.01, 0.1, 1, 10],
#      "min_samples_leaf":[ 10, 15, 25, 50],
#      "max_features":['auto', 'sqrt', 'log2', None],
#     "max_depth":[None, 5, 10, 20]},
]

In [35]:
ppp= r'^[a-zA-Z]+'
tabela_wynikow=pd.DataFrame(index = [list(range(7))], 
                            columns=["nazwa_modelu", "parametry", "accuracy", "f1","precision", "recall", "AUC" ])
for model, param_grid in zip(models, params):
    optimizer = GridSearchCV(model, param_grid, cv=10, n_jobs=-1)
    optimizer.fit(X_train_svd, y_train)
    
    #do ladnego podsumowania:
    nazwa_modelu = re.findall(ppp,str(optimizer.best_estimator_))
    i=models.index(model)
    tabela_wynikow["nazwa_modelu"][[i]]=nazwa_modelu
    tabela_wynikow["parametry"][[i]]=str(optimizer.best_params_)
    tabela_wynikow["accuracy"][[i]]=accuracy_score(y_test, optimizer.best_estimator_.predict(X_test_svd))
    tabela_wynikow["f1"][[i]]=f1_score(y_test, optimizer.best_estimator_.predict(X_test_svd))
    tabela_wynikow["precision"][[i]]=precision_score(y_test, optimizer.best_estimator_.predict(X_test_svd))
    tabela_wynikow["recall"][[i]]=recall_score(y_test, optimizer.best_estimator_.predict(X_test_svd))
    tabela_wynikow["AUC"][[i]]=roc_auc_score(y_test, optimizer.best_estimator_.predict(X_test_svd))   


In [38]:
tabela_wynikow[:3]

,nazwa_modelu,parametry,accuracy,f1,precision,recall,AUC
0,LogisticRegression,"{'C': 1, 'penalty': 'l2'}",0.81,0.802768,0.822695,0.783784,0.809655
1,SVC,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}",0.796667,0.789655,0.806338,0.773649,0.796364
2,RandomForestClassifier,"{'criterion': 'entropy', 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 10, 'n_estimators': 200}",0.741667,0.729494,0.754513,0.706081,0.741198


#### 4. Podsumowanie

Przetestowano 3 różne modele, z różnymi wariantami parametrów.
Powyższa tabela pokazuje najpopularniejsze miary służące do oceny modeli. 
Na podstawie tych danych najlepiej poradziła sobie regresja logistyczna.